In [ ]:
!pip install 'farm-haystack[elasticsearch]'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Install Dependencies
# ! pip install git+https://github.com/deepset-ai/haystack.git
! pip install urllib3==1.25.4
! pip install pandas
! pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached urllib3-1.25.4-py2.py3-none-any.whl (125 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.15
    Uninstalling urllib3-1.26.15:
      Successfully uninstalled urllib3-1.26.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.12.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 3.20.2 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:

# Install Elastic Search for Document Retriver
! wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
! tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
! chown -R daemon:daemon elasticsearch-7.9.2

import os
from subprocess import Popen, PIPE, STDOUT
es_server = Popen(['elasticsearch-7.9.2/bin/elasticsearch'], stdout=PIPE, stderr=STDOUT, preexec_fn=lambda: os.setuid(1))
# wait until ES has started
! sleep 30

In [ ]:
%%bash
curl -sX GET "localhost:9200/"

{
  "name" : "232f63c30555",
  "cluster_name" : "elasticsearch",
  "cluster_uuid" : "iv6XcIjxSIW8NxvbESQ5tA",
  "version" : {
    "number" : "7.9.2",
    "build_flavor" : "default",
    "build_type" : "tar",
    "build_hash" : "d34da0ea4a966c4e49417f2da2f244e3e97b4e6e",
    "build_date" : "2020-09-23T00:45:33.626720Z",
    "build_snapshot" : false,
    "lucene_version" : "8.6.2",
    "minimum_wire_compatibility_version" : "6.8.0",
    "minimum_index_compatibility_version" : "6.0.0-beta1"
  },
  "tagline" : "You Know, for Search"
}


In [ ]:
# Import libraries and dependencies 
import logging
import os
import re
import pandas as pd
from haystack.document_stores import ElasticsearchDocumentStore

In [ ]:

# Define the logger
logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

In [ ]:
# Get the host where Elasticsearch is running, default to localhost
host = os.environ.get("ELASTICSEARCH_HOST", "localhost")
document_store = ElasticsearchDocumentStore(host=host, username="", password="", index="document")

In [ ]:
# Mount the google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import xml.etree.ElementTree as ET

# Load XML file
tree = ET.parse('salesforce.stackexchange.com/Posts.xml')

# Get the root element
root = tree.getroot()

# Initialize dictionaries
questions = {}
answers = {}

# Iterate over each 'row' element
for row in root.iter('row'):
    post_type_id = row.attrib.get('PostTypeId')
    if post_type_id == "1":
        # This is a question
        question_id = row.attrib.get('Id')
        title = row.attrib.get('Title')
        body = row.attrib.get('Body')
        score = row.attrib.get('Score')
        questions[question_id] = {'title': title, 'body': body, 'score' : score}
    elif post_type_id == "2":
        # This is an answer
        answer_id = row.attrib.get('Id')
        question_id = row.attrib.get('ParentId')
        text = row.attrib.get('Body')
        text = text.strip().lower()
        text = re.sub("<.*?>", " ", text)
        score = row.attrib.get('Score')
        answer_dict = {'answer_id': answer_id, 'body': text, 'score' : score}
        if question_id in answers:
            answers[question_id].append(answer_dict)
        else:
            answers[question_id] = [answer_dict]


In [ ]:
questions["1"]

{'title': 'Live Agent ports/ip addresses',
 'body': '<p>What ports and IP addresses does Live Agent need for an agent to be able to connect to the chat from inside a corporate network?</p>\n',
 'score': '2'}

In [ ]:
answers["1"]

[{'answer_id': '49723',
  'body': ' you can find the endpoint for live agent of your org on the following page:  \n\n  setup &gt; customize &gt; live agent &gt; settings\n  \n\n and as this https endpoint it uses 443 port ',
  'score': '1'}]

In [ ]:
print("Total posts : ", len(questions.keys()))

Total posts :  129096


In [ ]:
# importing required libraries
import re
import nltk
import time
from nltk.corpus import movie_reviews
import numpy as np
import string
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from tqdm import tqdm
from sklearn.utils import shuffle
nltk.download('movie_reviews')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
porter_stemmer = PorterStemmer()
punctuations = set(string.punctuation)
stopwords=stopwords.words('english')
stopwords.remove('no')
stopwords.remove('nor')
stopwords.remove('not')
for item in stopwords:
    if "n't" in item:
        stopwords.remove(item)

### preprocess the text data for BM25:
* Convert string to lower cases.
* Remove tailing spaces, special characters, HTML tags and \n, \t tokens.
* Replace License plate number, ID number and email with respective unique identifiers
* Remove Stop words and apply porter stemmer on processed text. 

In [ ]:
def process_sentence(sentence):
    if sentence == np.NaN:
      return
    sentence = sentence.lower()
    # Remove trailing spaces
    sentence = sentence.strip()
    
    # Remove HTMl tages from the text
    sentence = re.sub("<.*?>", " ", sentence)
    
     # Remove special characters and numbers in the string
    sentence = re.sub(r'[^\w\s]',' ',sentence)
    sentence = re.sub(" \d+", " ",sentence)
    
    # Remove \n and \t in sentence
    sentence = re.sub("\n", "", sentence)
    sentence = re.sub("\t", "", sentence)
    sentence = ''.join(ch for ch in sentence if ch not in punctuations)
    
    # Remove stop words and word has length less than 3
    tokenized_sentence = sentence.split(" ")
    processed_review = [token for token in tokenized_sentence if token and token not in stopwords and len(token) >= 3 and not token.isdigit()]
    # Get the stem the word
    processed_review = " ".join(list(map(lambda x: porter_stemmer.stem(x), processed_review)))
#     processed_review = " ".join(processed_review)
    return processed_review

In [ ]:
start_time = time.time()
bm25_processed_docs = []
for item in questions:
  title = questions[item]["title"]
  body = questions[item]["body"]
  processed_text = process_sentence(title + body)
  score = questions[item]["score"]
  bm25_processed_docs.append({"content" : processed_text, "meta": {"question_id": item, "score": score}})
end_time = time.time()
print("Time taken to prepocess the text data for BM25 model : ", end_time - start_time)

Time taken to prepocess the text data for BM25 model :  346.55640029907227


In [ ]:
bm25_processed_docs[0]

{'content': 'live agent port address port address live agent need agent abl connect chat insid corpor network',
 'meta': {'question_id': '1', 'score': '2'}}

In [ ]:
# Define the elastic search document store
import os
from haystack.document_stores import ElasticsearchDocumentStore

# Get the host where Elasticsearch is running, default to localhost
host = os.environ.get("ELASTICSEARCH_HOST", "localhost")
elastic_document_store = ElasticsearchDocumentStore(host=host, username="", password="", index="document-dim", similarity="cosine", recreate_index=True)

INFO:haystack.document_stores.search_engine:Index 'label' deleted.


In [ ]:
# Write the processed document to the elastic search document store
start_time = time.time()
elastic_document_store.write_documents(bm25_processed_docs)
print("Time taken to write and index the documents in Elastic search datastore for the BM25 model : ", time.time() - start_time)

Time taken to write and index the documents in Elastic search datastore for the BM25 model :  273.5715751647949


In [ ]:
print(f"Loaded {elastic_document_store.get_document_count()} documents")

Loaded 129093 documents


In [ ]:
elastic_document_store.embedding_dim

768

In [ ]:
# Define the BM25 Retriever
from haystack.nodes import BM25Retriever
bm25_retriever = BM25Retriever(elastic_document_store)

In [ ]:
for item in bm25_retriever.retrieve(process_sentence("What ports and IP addresses does Live Agent need for an agent to be able to connect to the chat from inside a corporate network."), top_k=10):
  print(item.to_dict())

{'content': 'live agent port address port address live agent need agent abl connect chat insid corpor network', 'content_type': 'text', 'score': 0.9999768834238804, 'meta': {'score': '2', 'question_id': '1'}, 'id_hash_keys': ['content'], 'embedding': None, 'id': '60e9cf7b2850c2e8cebf71a94d82d6a1'}
{'content': 'live agent save chat agent close connect notic agent close chat user not abl save chat redirect post chat page offici way notifi user agent close connect', 'content_type': 'text', 'score': 0.9958469796657856, 'meta': {'score': '1', 'question_id': '139477'}, 'id_hash_keys': ['content'], 'embedding': None, 'id': '207fae84d964adb92d23cc6c14374301'}
{'content': 'live agent chat window custom need custom insid live agent chat window want understand possibl chatalertmessag dismisslabel close agent not avail particular time span want redirect url agent not avail possibl disabl agent type chat screen want show alert messag screen custom repli sec still chat end sec repli', 'content_type'

In [ ]:
# Load the test data for evaluating the search engine
test_data = pd.read_csv("salesforce.stackexchange.com/Test data.csv")

In [ ]:
test_data.head()

,Question,Question Id
0,I am transitioning away from my previous CRM system and searching for user-f...,10
1,I am in the process of creating an email template with an HTML letterhead fo...,13
2,How much can I change using point and click configuration,52
3,My objective is to find the simplest method to generate a lead using email.,75
4,Has anyone been able to solve this problem for large batches of records?,109


In [ ]:
# Evaluating the search engine by true positive rate and false negative score
def evaluate_retriever(retriever, test_data, no_of_documents):
  pred_labels = []
  true_label_count = 0
  false_label_count = 0
  for _, row in test_data.iterrows():
    true_label = str(row["Question Id"])
    pred_documents = []
    for item in  retriever.retrieve(process_sentence(row["Question"]), top_k=no_of_documents):
      question_id = item.to_dict()["meta"]["question_id"]
      pred_documents.append(question_id)
    if true_label in pred_documents:
      true_label_count += 1
    else:
      false_label_count += 1
    pred_labels.append(pred_documents)
  return pred_labels, true_label_count, false_label_count

In [ ]:
pred_labels, true_label_count, false_label_count = evaluate_retriever(bm25_retriever, test_data, 10)

In [ ]:
print("Recall of search engine using BM25 embeddings as the index is : ", (true_label_count/(true_label_count + false_label_count)))

Recall of search engine using BM25 embeddings as the index is :  0.86


## DensePassageRetriever


In [ ]:
import os
from haystack.document_stores import ElasticsearchDocumentStore
# Get the host where Elasticsearch is running, default to localhost
host = os.environ.get("ELASTICSEARCH_HOST", "localhost")
elastic_document_store_2 = ElasticsearchDocumentStore(host=host, username="", password="", index="document-dim-2", similarity="cosine", recreate_index=True)

INFO:haystack.document_stores.search_engine:Index 'label' deleted.


In [ ]:
# Convert the document to lower case and remove the trailing spaces.
# Remove the HTML Tags
processed_data = []

for item in questions:
  title = questions[item]["title"]
  body = questions[item]["body"]
  text = title + body
  text = text.strip().lower()
  sentence = re.sub("<.*?>", " ", text)
  score = questions[item]["score"]
  processed_data.append({"content" : sentence, "meta": {"question_id": item, "score": score}})

In [ ]:
start_time = time.time()
elastic_document_store_2.write_documents(processed_data)
print("Time taken to write the documents in Elastic search datastore for the DenseParser model : ", time.time() - start_time)

Time taken to write the documents in Elastic search datastore for the DenseParser model :  273.07565116882324


In [ ]:
from haystack.nodes import DensePassageRetriever

dense_parser_retriever = DensePassageRetriever(
    document_store=elastic_document_store_2,
    query_embedding_model="facebook/dpr-question_encoder-single-nq-base",
    passage_embedding_model="facebook/dpr-ctx_encoder-single-nq-base")

INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1


/usr/local/lib/python3.10/dist-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
INFO:haystack.modeling.model.language_model:Auto-detected model language: english


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizerFast'.


INFO:haystack.modeling.model.language_model:Auto-detected model language: english


In [ ]:
start_time = time.time()
elastic_document_store_2.update_embeddings(dense_parser_retriever)
print("Time taken to update index of documents in Elastic search datastore using DenseParser model : ", time.time() - start_time)

INFO:haystack.document_stores.search_engine:Updating embeddings for all 129094 docs ...


Updating embeddings:   0%|          | 0/129094 [00:00<?, ? Docs/s]

Create embeddings:   0%|          | 0/10000 [00:00<?, ? Docs/s]

Create embeddings:   0%|          | 0/10000 [00:00<?, ? Docs/s]

Create embeddings:   0%|          | 0/10000 [00:00<?, ? Docs/s]

...

Create embeddings:   0%|          | 0/10000 [00:00<?, ? Docs/s]

Create embeddings:   0%|          | 0/10000 [00:00<?, ? Docs/s]

Create embeddings:   0%|          | 0/10000 [00:00<?, ? Docs/s]

Create embeddings:   0%|          | 0/9104 [00:00<?, ? Docs/s]

Time taken to update index of documents in Elastic search datastore using DenseParser model :  2656.923171043396


In [ ]:
for item in dense_parser_retriever.retrieve("What ports and IP addresses does Live Agent need for an agent to be able to connect to the chat from inside a corporate network.", top_k=10):
  print(item.to_dict()["meta"])

{'score': '2', 'question_id': '1'}
{'score': '1', 'question_id': '117280'}
{'score': '1', 'question_id': '256338'}
{'score': '2', 'question_id': '17344'}
{'score': '1', 'question_id': '304521'}
{'score': '0', 'question_id': '210149'}
{'score': '1', 'question_id': '295262'}
{'score': '0', 'question_id': '384621'}
{'score': '1', 'question_id': '186258'}
{'score': '1', 'question_id': '186301'}


In [ ]:
# Evaluating the reteriver by true positive rate and false negative score
def evaluate_retriever(retriever, test_data, no_of_documents):
  pred_labels = []
  true_label_count = 0
  false_label_count = 0
  for _, row in test_data.iterrows():
    true_label = str(row["Question Id"])
    pred_documents = []
    for item in  retriever.retrieve(row["Question"], top_k=no_of_documents):
      document_name = item.to_dict()["meta"]["question_id"]
      pred_documents.append(document_name)
    if true_label in pred_documents:
      true_label_count += 1
    else:
      false_label_count += 1
    pred_labels.append(pred_documents)
  return pred_labels, true_label_count, false_label_count


In [ ]:
# Evaluating the Dense parser retriever
pred_labels, true_label_count, false_label_count = evaluate_retriever(dense_parser_retriever, test_data, 10)

In [ ]:
print("Recall of Dense parser Reteriver is : ", (true_label_count/(true_label_count + false_label_count)))

Recall of Dense parser Reteriver is :  0.21


# Embedding Retriever

In [ ]:
import os
from haystack.document_stores import ElasticsearchDocumentStore

# Get the host where Elasticsearch is running, default to localhost
host = os.environ.get("ELASTICSEARCH_HOST", "localhost")
elastic_document_store_3 = ElasticsearchDocumentStore(host=host, username="", password="", index="document-dim-2", similarity="cosine", recreate_index=True, embedding_dim=384)

In [ ]:
start_time = time.time()
elastic_document_store_3.write_documents(processed_data)
print("Time taken to write the documents in Elastic search datastore for the Embedding model : ", time.time() - start_time)

Time taken to write the documents in Elastic search datastore for the Embedding model :  271.93708086013794


In [ ]:
from haystack.nodes import EmbeddingRetriever

embedding_retriever = EmbeddingRetriever(
    document_store=elastic_document_store_3,
    embedding_model="sentence-transformers/all-MiniLM-L6-v2",
    model_format="sentence_transformers",
)
# Important:
# Now that we initialized the Retriever, we need to call update_embeddings() to iterate over all
# previously indexed documents and update their embedding representation.
# While this can be a time consuming operation (depending on the corpus size), it only needs to be done once.
# At query time, we only need to embed the query and compare it to the existing document embeddings, which is very fast.
elastic_document_store_3.update_embeddings(embedding_retriever)

INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
INFO:haystack.nodes.retriever.dense:Init retriever using embeddings of model sentence-transformers/all-MiniLM-L6-v2


Updating embeddings:   0%|          | 0/129094 [00:00<?, ? Docs/s]

Batches:   0%|          | 0/313 [00:00<?, ?it/s]

Batches:   0%|          | 0/313 [00:00<?, ?it/s]

Batches:   0%|          | 0/313 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/313 [00:00<?, ?it/s]

Batches:   0%|          | 0/313 [00:00<?, ?it/s]

Batches:   0%|          | 0/313 [00:00<?, ?it/s]

Batches:   0%|          | 0/313 [00:00<?, ?it/s]

Batches:   0%|          | 0/313 [00:00<?, ?it/s]

In [ ]:
for item in embedding_retriever.retrieve("How to create Purchase Report by Vendor grouped by Class.", top_k=10):
  print(item.to_dict())

In [ ]:
pred_labels, true_label_count, false_label_count = evaluate_retriever(embedding_retriever, test_data, 10)

In [ ]:
print("Recall of Embeddding Reteriver is : ", (true_label_count/(true_label_count + false_label_count)))